In [ ]:
# TODO
# 1 match the names between elo and fixtures
# 2 simulate season
### a) calculate probs of win/draw/loss
### b) update elo afterwards
# 3 formulate draw rules
# 4 scrape outrights odds

# european leagues with seasons not ending around May: Finland, Ireland, Norway, Sweden
# american leagues -||-: Brazil, Canada, Ecuador, USA, Uruguay
# asian leagues -||-: Japan, Korea et al.

In [1]:
import copy
from datetime import datetime
import json
import math
import os
from pathlib import Path
import random

from dotenv import load_dotenv
import pandas as pd
import requests

In [2]:
load_dotenv()

API_TOKEN = os.getenv('X-RapidAPI-Key')

In [3]:
def download_elo_data() -> None:
    today = datetime.today().strftime('%Y-%m-%d')
    df = pd.read_csv(f"http://api.clubelo.com/{today}")
    Path("data/elo").mkdir(parents=True, exist_ok=True)
    df = df[['Rank', 'Club', 'Country', 'Level', 'Elo']]
    today = today.replace('-', '')
    df.to_csv(f"data/elo/{today}.csv", index=False)

In [4]:
def api_get_leagues() -> None:
    url = "https://api-football-v1.p.rapidapi.com/v3/leagues"

    params = {"current": "true"}

    headers = {
        "X-RapidAPI-Key": API_TOKEN,
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)
    print(response.status_code)

    if response.json()['paging']['total'] != 1:
        raise Exception("Error: multiple pages of leagues")

    Path("data/api").mkdir(parents=True, exist_ok=True)
    with open("data/api/leagues.json", "w") as f:
        json.dump(response.json(), f)

In [5]:
def api_get_fixtures_for_league(league_id: str, season: str) -> None:
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
    params = {"league": league_id, "season": season}

    headers = {
        "X-RapidAPI-Key": API_TOKEN,
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.json()['results'] == 0:
        raise Exception("No results found.")

    if response.json()['paging']['total'] != 1:
        raise Exception("Error: multiple pages of leagues")

    Path("data/api").mkdir(parents=True, exist_ok=True)
    with open(f"data/api/fixtures_{league_id}_{season}.json", "w") as f:
        json.dump(response.json(), f)

In [6]:
def api_get_standings_for_league() -> None:
    pass

In [22]:
def find_league_id(country_code: str, league_name: str) -> str:
    with open("data/api/leagues.json", "r") as f:
        leagues = json.load(f)
        for league in leagues['response']:
            if league['country']['code'] == country_code and league['league']['type'] == 'League' and league['league']['name'] == league_name:
                return league['league']['id']

assert find_league_id("NO", "Eliteserien") == 103
assert find_league_id("PL", "Ekstraklasa") == 106
assert find_league_id("SE", "Allsvenskan") == 113

In [21]:
def get_team_names_from_elo(elo_country_code: str) -> list:
    df = pd.read_csv(find_latest_elo_file())
    df = df[(df['Country'] == elo_country_code) & (df['Level'] == 1)]
    return sorted(df['Club'].tolist())

assert 'Elfsborg' in get_team_names_from_elo('SWE')

In [13]:
def find_latest_elo_file() -> str:
    elo_files = os.listdir("data/elo")
    return f'data/elo/{sorted(elo_files)[-1]}'

In [ ]:
def simulate_results() -> None:
    pass